In [1]:
import os
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as feature
from matplotlib.colors import LinearSegmentedColormap, TwoSlopeNorm
import numpy as np

In [2]:
directory = "swd_files"
file_paths = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(".nc")]
swd_arrays = [xr.open_dataarray(file) for file in file_paths]

In [3]:
swd_climatology_path = "swd_climatology_files/2024_01_08_14Z-2024_01_08_23Z.nc"
swd_climatology = xr.open_dataarray(swd_climatology_path)

#--- Filter to only positive values
swd_climatology_pos = swd_climatology.where(swd_climatology > 0, other=np.nan)

In [4]:
output_directory = "swd_file_clim_diff"

for i, swd_array in enumerate(swd_arrays):
    # Subtract the median DataArray
    result = swd_array - swd_climatology_pos

    # Define an output filename
    output_filename = f"{output_directory}/result_{file_paths[i][24:38]}.nc"

    # Save the result as a NetCDF file
    result.to_netcdf(output_filename)

    print(f"Saved: {output_filename}")

/var/folders/pd/x01_y6d52zj47w6_m5_h_8f80000gr/T/ipykernel_57935/1504723129.py:11: SerializationWarning: saving variable y with floating point data as an integer dtype without any _FillValue to use for NaNs
  result.to_netcdf(output_filename)
/var/folders/pd/x01_y6d52zj47w6_m5_h_8f80000gr/T/ipykernel_57935/1504723129.py:11: SerializationWarning: saving variable x with floating point data as an integer dtype without any _FillValue to use for NaNs
  result.to_netcdf(output_filename)
/var/folders/pd/x01_y6d52zj47w6_m5_h_8f80000gr/T/ipykernel_57935/1504723129.py:11: SerializationWarning: saving variable y with floating point data as an integer dtype without any _FillValue to use for NaNs
  result.to_netcdf(output_filename)
/var/folders/pd/x01_y6d52zj47w6_m5_h_8f80000gr/T/ipykernel_57935/1504723129.py:11: SerializationWarning: saving variable x with floating point data as an integer dtype without any _FillValue to use for NaNs
  result.to_netcdf(output_filename)
/var/folders/pd/x01_y6d52zj4

Saved: swd_file_clim_diff/result_2024_01_08_16Z.nc
Saved: swd_file_clim_diff/result_2024_01_08_17Z.nc
Saved: swd_file_clim_diff/result_2024_01_08_15Z.nc
Saved: swd_file_clim_diff/result_2024_01_08_14Z.nc
Saved: swd_file_clim_diff/result_2024_01_08_19Z.nc
Saved: swd_file_clim_diff/result_2024_01_08_18Z.nc
Saved: swd_file_clim_diff/result_2024_01_08_23Z.nc
Saved: swd_file_clim_diff/result_2024_01_08_22Z.nc
Saved: swd_file_clim_diff/result_2024_01_08_20Z.nc
Saved: swd_file_clim_diff/result_2024_01_08_21Z.nc


/var/folders/pd/x01_y6d52zj47w6_m5_h_8f80000gr/T/ipykernel_57935/1504723129.py:11: SerializationWarning: saving variable y with floating point data as an integer dtype without any _FillValue to use for NaNs
  result.to_netcdf(output_filename)
/var/folders/pd/x01_y6d52zj47w6_m5_h_8f80000gr/T/ipykernel_57935/1504723129.py:11: SerializationWarning: saving variable x with floating point data as an integer dtype without any _FillValue to use for NaNs
  result.to_netcdf(output_filename)
/var/folders/pd/x01_y6d52zj47w6_m5_h_8f80000gr/T/ipykernel_57935/1504723129.py:11: SerializationWarning: saving variable y with floating point data as an integer dtype without any _FillValue to use for NaNs
  result.to_netcdf(output_filename)
/var/folders/pd/x01_y6d52zj47w6_m5_h_8f80000gr/T/ipykernel_57935/1504723129.py:11: SerializationWarning: saving variable x with floating point data as an integer dtype without any _FillValue to use for NaNs
  result.to_netcdf(output_filename)
/var/folders/pd/x01_y6d52zj4

In [5]:
input_directory = "swd_file_clim_diff"
output_directory = "swd_file_clim_diff/plots"
os.makedirs(output_directory, exist_ok=True)
file_paths = [os.path.join(input_directory, f) for f in os.listdir(input_directory) if f.endswith(".nc")]

In [6]:
#--- US Southwest
latitude_north = 40.5
latitude_south = 27.5
longitude_west = -123
longitude_east = -100

In [7]:
for file_path in file_paths:
    # Load the DataArray
    swd_result = xr.open_dataarray(file_path)

    # Create the plot
    projection = ccrs.PlateCarree(central_longitude=0)
    fig, ax = plt.subplots(1, figsize=(12, 12), subplot_kw={'projection': projection})

    # Define custom colormap and normalization
    cmap = LinearSegmentedColormap.from_list('custom_cmap', ['grey', 'white', 'red'])
    norm = TwoSlopeNorm(vmin=-12, vcenter=0, vmax=3)
    levels = np.linspace(-12, 3, 31)

    c = ax.contourf(swd_result.lon, swd_result.lat, swd_result, cmap=cmap, extend='both', norm=norm, levels=levels)

    # Add colorbar
    clb = plt.colorbar(c, shrink=0.4, pad=0.02, ax=ax)
    clb.ax.tick_params(labelsize=15)
    clb.set_label('(K)', fontsize=15)

    # Set map extent (adjust longitude and latitude bounds as needed)
    ax.set_extent([longitude_west, longitude_east, latitude_south, latitude_north], crs=ccrs.PlateCarree())

    datetime_str = file_path[24:38]
    ax.set_title("GOES BTD (12.3 μm - 10.3 μm) \n(" + datetime_str + ")", 
                 fontsize=20, pad=10)

    ax.coastlines(resolution='50m', color='black', linewidth=1)
    ax.add_feature(feature.STATES, zorder=100, edgecolor='#000', facecolor='none', linewidth=0.5)

    # Save the figure with a dynamic filename
    output_filename = os.path.join(output_directory, f"{os.path.basename(file_path).replace('.nc', '.png')}")
    fig.savefig(output_filename, dpi=200, bbox_inches='tight')

    # Close the figure to free memory
    plt.close(fig)

    print(f"Saved plot: {output_filename}")

Saved plot: swd_file_clim_diff/plots/result_2024_01_08_21Z.png
Saved plot: swd_file_clim_diff/plots/result_2024_01_08_20Z.png
Saved plot: swd_file_clim_diff/plots/result_2024_01_08_22Z.png
Saved plot: swd_file_clim_diff/plots/result_2024_01_08_23Z.png
Saved plot: swd_file_clim_diff/plots/result_2024_01_08_18Z.png
Saved plot: swd_file_clim_diff/plots/result_2024_01_08_19Z.png
Saved plot: swd_file_clim_diff/plots/result_2024_01_08_14Z.png
Saved plot: swd_file_clim_diff/plots/result_2024_01_08_15Z.png
Saved plot: swd_file_clim_diff/plots/result_2024_01_08_17Z.png
Saved plot: swd_file_clim_diff/plots/result_2024_01_08_16Z.png
